In [2]:
from _shared import *

### Задача  1. Оценка разных статистик

В лекции мы выяснили, что в эксперименте «Refactoring backend» в экспериментальной группе среднее время загрузки увеличилось, а 99% квантиль уменьшился. Проверьте значимость отличий других квантилей.

Данные эксперимента «Refactoring backend»:

`2022-04-13/2022-04-13T12_df_web_logs.csv` и `2022-04-13/experiment_users.csv`.

Эксперимент проводился с __2022-04-05__ по __2022-04-12__.

Измерения времени обработки запросов считаем независимыми. 
При проверке используйте нормальный доверительный интервал.

Выберете верные утверждения:
- Квантиль 0.70 статистически значимо изменился
- Квантиль 0.74 статистически значимо изменился
- Квантиль 0.78 статистически значимо изменился
- Квантиль 0.82 статистически значимо изменился
- Квантиль 0.86 статистически значимо изменился
- Квантиль 0.90 статистически значимо изменился
- Квантиль 0.95 статистически значимо изменился
- Квантиль 0.99 статистически значимо изменился
- Квантиль 0.999 статистически значимо изменился
- Квантиль 0.9999 статистически значимо изменился

In [14]:
# df_web_logs = read_from_database('2022-04-13/2022-04-13T12_df_web_logs.csv') #Fails due to the bad naming.
df_web_logs = pd.read_csv('./data/2022-04-13T12_df_web_logs.csv', parse_dates=[2])

df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [16]:
# experiment_users = read_from_database('experiment_users.csv') #Different.
experiment_users = pd.read_csv('./data/experiment_users.csv')

experiment_users.head()

,user_id,pilot
0,0ffc65,0
1,b962b9,0
2,7ea63f,0
3,7f9a61,0
4,459e55,0
